### Importando classes

In [157]:
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

### Instância


In [201]:
services = webdriver.ChromeService()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=services, options=options)

In [202]:
url = 'https://transparencia.ac.gov.br/#/servidor-publico'
driver.get(url)

In [203]:
#seletores das combobox


combobox_situacao = driver.find_elements(by=By.CLASS_NAME, value="conteudo-bloco-filtro-select")[0]
select = Select(combobox_situacao)
select.select_by_visible_text("Ativo")

combobox_periodo_mes = driver.find_elements(by=By.CLASS_NAME, value="conteudo-bloco-filtro-select")[1]
select = Select(combobox_periodo_mes)
select.select_by_visible_text("Janeiro")

combobox_periodo_ano = driver.find_elements(by=By.CLASS_NAME, value="conteudo-bloco-filtro-select")[2]
select = Select(combobox_periodo_ano)
select.select_by_value("2023")

#click no botão FILTRAR
driver.find_element(by=By.TAG_NAME, value="button").click()

#esperando a página responder
#time.sleep(50)

In [194]:
#pegando a tabela e as linhas

tabela = driver.find_element(By.XPATH, "//table")
colunas = tabela.find_elements(By.XPATH, "//thead/tr/th")
linhas = tabela.find_elements(By.XPATH, "//tbody/tr")

nomes_colunas  = [header.text for header in colunas]
dados_linhas = [i.text for i in linhas]

grupos_palavras = ['APOSENTADO', 'EM EXERCÍCIO', 'EM FÉRIAS', 'PENSIONISTA', 'AFASTADO/LICENCIADO']

caminho_arquivo= 'cargos_servidor.csv'

df = pd.read_csv(caminho_arquivo, header=None, names=['Cargos'], quoting=3)

# Extrair a coluna 'Cargos' como uma lista
lista_cargos = df['Cargos'].tolist()
lista_cargos

['MEDICO',
 'APOSENTADOS',
 'PROFESSOR',
 'AGENTE',
 'TEC. EM',
 'TECNICO',
 'MEDICO',
 '1* ',
 '2* ',
 '3* ',
 'PENSIONISTA',
 'SOLDADO',
 'ESPEC.',
 'ALUNO',
 'DESENHISTA',
 'SUB-TENENTE',
 'APOIO',
 'ENFERMEIRO',
 'VIGIA',
 'MOTORISTA',
 'AUXILIAR',
 'BIOMEDICO',
 'CABO']

In [184]:
def adicionar_espacos(texto, lista_palavras):
    for palavra in lista_palavras:
        padrao = re.compile(rf'\b{re.escape(palavra)}\b')
        texto = re.sub(padrao, f' {palavra}', texto)
    return texto

In [200]:
# Criar uma lista de dicionários
registros = []

for linha in dados_linhas:

    linha = adicionar_espacos(linha, grupos_palavras)
    linha = adicionar_espacos(linha, lista_cargos)
    # Dividir a linha em partes usando o método split
    partes = linha.split("  ")

    # Criar um dicionário vazio para armazenar os dados desta linha
    registro = {}

    # Iterar sobre as colunas e adicionar os valores correspondentes
    for i in range(len(nomes_colunas)):
        coluna = nomes_colunas[i]
        valor = partes[i] if i < len(partes) else ""  # Tratar o caso em que a linha não tem todas as colunas
        registro[coluna] = valor

    # Adicionar o dicionário à lista de registros
    registros.append(registro)


# Exibir os registros
for registro in registros:
   print(registro)

df = pd.DataFrame(registros)
df



{'Nome': 'AARON RODRIGUES GOMES', 'Situação': 'EM EXERCÍCIO', 'Cargo': ' MEDICO', 'Bruto': '13.907,13', 'Desconto': '3.453,59', 'Líquido': '10.453,54'}
{'Nome': 'ABDENAGO ALVES PEREIRA FILHO', 'Situação': 'APOSENTADO', 'Cargo': 'APOSENTADOS - MEDIA', 'Bruto': '4.975,59', 'Desconto': '77,50', 'Líquido': '4.898,09'}
{'Nome': 'ABDENAGO ALVES PEREIRA FILHO', 'Situação': 'APOSENTADO', 'Cargo': 'PROFESSOR P2 30H - CLASSE II', 'Bruto': '4.412,60', 'Desconto': '2.401,64', 'Líquido': '2.010,96'}
{'Nome': 'ABDENEGO DE PAULA CAMILLO', 'Situação': 'EM EXERCÍCIO', 'Cargo': 'CABO', 'Bruto': '6.354,06', 'Desconto': '3.658,45', 'Líquido': '2.695,61'}
{'Nome': 'ABDIAS COSTA DA SILVA', 'Situação': 'EM EXERCÍCIO', 'Cargo': 'PROFESSOR N SUPERIOR-P2 CONT TEMP-25H', 'Bruto': '5.415,70', 'Desconto': '699,48', 'Líquido': '4.716,22'}
{'Nome': 'ABDIAS PINHEIRO MAIA', 'Situação': 'EM EXERCÍCIO', 'Cargo': 'AGENTE DE SAUDE PUBLICA', 'Bruto': '3.680,63', 'Desconto': '415,00', 'Líquido': '3.265,63'}
{'Nome': 'ABDIAS

,Nome,Situação,Cargo,Bruto,Desconto,Líquido
0,AARON RODRIGUES GOMES,EM EXERCÍCIO,MEDICO,"13.907,13","3.453,59","10.453,54"
1,ABDENAGO ALVES PEREIRA FILHO,APOSENTADO,APOSENTADOS - MEDIA,"4.975,59","77,50","4.898,09"
2,ABDENAGO ALVES PEREIRA FILHO,APOSENTADO,PROFESSOR P2 30H - CLASSE II,"4.412,60","2.401,64","2.010,96"
3,ABDENEGO DE PAULA CAMILLO,EM EXERCÍCIO,CABO,"6.354,06","3.658,45","2.695,61"
4,ABDIAS COSTA DA SILVA,EM EXERCÍCIO,PROFESSOR N SUPERIOR-P2 CONT TEMP-25H,"5.415,70","699,48","4.716,22"
5,ABDIAS PINHEIRO MAIA,EM EXERCÍCIO,AGENTE DE SAUDE PUBLICA,"3.680,63","415,00","3.265,63"
6,ABDIAS RODRIGUES DE ARAUJO,EM EXERCÍCIO,TEC. EM DEFESA AGROPECUARIA E FLORESTAL,"24.490,58","11.010,53","13.480,05"
7,ABEGNO GADELHA DE MENDONCA,EM EXERCÍCIO,MEDICO,"17.326,32","4.393,87","12.932,45"
8,ABEL AMORIM DE OLIVEIRA,EM EXERCÍCIO,2* SARGENTO,"8.499,42","1.991,55","6.507,87"
9,ABEL APOLINARIO DA SILVA,EM EXERCÍCIO,1* TENENTE,"15.741,77","4.241,17","11.500,60"


In [242]:
driver.find_element(By.XPATH, "//a[@aria-label='Próximo page']").click()

#[i.text for i in proximo]